In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from dotenv import load_dotenv
import os

# 1. 출력 스키마 정의
response_schemas = [
    ResponseSchema(name="destination", description="여행지 (도시나 지역 이름)"),
    ResponseSchema(name="duration", description="여행 기간 (예: 2박 3일, 5일 등)"),
    ResponseSchema(name="budget", description="예산 (숫자 + 단위, 예: 30만원)"),
    ResponseSchema(name="rating", description="여행 추천도, 1~5점 중 하나"),
    ResponseSchema(name="activities", description="주요 활동 리스트 (여행 중 했던 활동들)")
]

# 2. 파서 생성
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 3. 프롬프트 정의
format_instructions = output_parser.get_format_instructions()

template = """
아래 여행 후기나 계획을 읽고 핵심 정보를 추출하세요.
반드시 {format_instructions} 형식으로 출력해야 합니다.

여행 텍스트:
{review}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["review"],
    partial_variables={"format_instructions": format_instructions},
)
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# 4. 모델 정의
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

# 5. 실행 함수
def analyze_trip(review: str):
    input_text = prompt.format(review=review)
    output = llm.predict(input_text)
    return output_parser.parse(output)

# 6. 테스트 입력
review_text = "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."

result = analyze_trip(review_text)
print(result)
